# Clasa Game

In [20]:
import numpy as np

class Game:
    MAX = 'x'
    MIN = '0'
    GOL = '.'
    
    MORI_POSIBILE = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8), (9, 10, 11), (12, 13, 14),
        (15, 16, 17), (18, 19, 20), (21, 22, 23),
        (0, 9, 21), (3, 10, 18), (6, 11, 15), (1, 4, 7), (16, 19, 22),
        (8, 12, 17), (5, 13, 20), (2, 14, 23)
    ]

    @classmethod
    def jucator_opus(self, jucator):
        return Game.MIN if jucator == Game.MAX else Game.MAX

# Clasa Nod

In [21]:
class Nod:
    def __init__(self, informatie=np.array([Game.GOL] * 24), parinte=None, jucator=Game.MAX, piese_de_pus=(0, 0)):
        self.informatie = informatie
        self.parinte = parinte
        self.jucator = jucator
        self.piese_de_pus = piese_de_pus
        self.euristica = 0

    def __repr__(self):
        return str(self.informatie)

    def __str__(self):
        
        b = self.informatie
        
        board = (
            f"    {b[0]}-----------{b[1]}-----------{b[2]}\n"
            f"    |           |           |\n"
            f"    |   {b[3]}-------{b[4]}-------{b[5]}   |\n"
            f"    |   |       |       |   |\n"
            f"    |   |   {b[6]}---{b[7]}---{b[8]}   |   |\n"
            f"    |   |   |       |   |   |\n"
            f"    {b[9]}---{b[10]}---{b[11]}       {b[12]}---{b[13]}---{b[14]}\n"
            f"    |   |   |       |   |   |\n"
            f"    |   |   {b[15]}---{b[16]}---{b[17]}   |   |\n"
            f"    |   |       |       |   |\n"
            f"    |   {b[18]}-------{b[19]}-------{b[20]}   |\n"
            f"    |           |           |\n"
            f"    {b[21]}-----------{b[22]}-----------{b[23]}\n"
        
        )
        return board

    def __eq__(self, other):
        return np.array_equal(self.informatie, other.informatie)

    def __lt__(self, other):
        if other is None:
            return True
        return self.euristica <= other.euristica
    
    def drumRadacina(self):
        ''' Calculeaza lista nodurilor de la radacina pana la nodul curent. '''
        if self.parinte is None:
            return np.array(self)
        return np.append(self.parinte.drumRadacina(), self)
    
    def succesori(self):
        ''' Calculeaza lista succesorilor directi ai starii curente.
        :return: lista starilor admisibile
        '''
        succesori = []
        n_x, n_0 = self.piese_de_pus
        next_piese = (n_x - 1, n_0) if self.jucator == Game.MAX else (n_x, n_0 - 1)

        for i in range(24):
            if self.informatie[i] == Game.GOL:
                info_noua = self.informatie.copy()
                info_noua[i] = self.jucator
                succesor = Nod(info_noua, self, Game.jucator_opus(self.jucator), next_piese)
                succesori.append(succesor)
        return succesori

    # Verifica daca jucatorul "simbol" a facut o moara.
    def check_moara(self, simbol):
        for moara in Game.MORI_POSIBILE:
            if all(self.informatie[pos] == simbol for pos in moara):
                return True
        return False

    # Verifica daca nodul curent este nod scop, adica daca este o stare finala a jocului
    def scop(self):
        if self.check_moara(Game.MAX):
            return Game.MAX
        if self.check_moara(Game.MIN):
            return Game.MIN
        if self.piese_de_pus[0] == 0 and self.piese_de_pus[1] == 0 and Game.GOL not in self.informatie:
            return 'remiza'
        return None

# Clasa ReteaBayesiana

In [22]:
from collections import defaultdict

class ReteaBayesiana:
    def __init__(self, game):
        self.game = game

    def evaluare_moara(self, nod, pos, moara):
        # Simuleaza mutarea in pozitia pos
        new_info = nod.informatie.copy()
        new_info[pos] = nod.jucator
        secventa = [new_info[p] for p in moara]
        
        # Verificam daca moara e blocata de adversar
        if Game.jucator_opus(nod.jucator) in secventa:
            return 0
        
        # Numaram piesele lui MAX din moara
        count = sum(1 for s in secventa if s == nod.jucator)
        if count == 3:
            return 0.9
        elif count > 0:
            return 0.3 * count
        return 0.1

    def evaluare_prob(self, nod, pos):
        prob = 0
        for moara in Game.MORI_POSIBILE:
            if pos in moara:
                prob += self.evaluare_moara(nod, pos, moara)
        return prob

    def select_move(self, nod):
        pozitii_libere = [i for i in range(24) if nod.informatie[i] == Game.GOL]
        best_pos = None
        best_prob = -float('inf')

        # Calculam probabilitatea pentru fiecare stare
        for pos in pozitii_libere:
            prob = self.evaluare_prob(nod, pos)
            if prob > best_prob:
                best_prob = prob
                best_pos = pos

        return best_pos


# Input si Rezolvare

In [23]:
stare_initiala = ['.'] * 24
stare_initiala[11] = 'x'
stare_initiala[12] = '0'
stare_initiala[13] = 'x'
stare_initiala[17] = '0'
stare_initiala[18] = 'x'
stare_initiala[19] = '0'
piese_de_pus = (6, 6)
nod_initial = Nod(np.array(stare_initiala), None, Game.MAX, piese_de_pus)

retea = ReteaBayesiana(Game)
mutare = retea.select_move(nod_initial)


print("-- Stare initiala --")
print(nod_initial)

print("-- Stare dupa mutare --")
nod_initial.informatie[mutare] = "x"
print(nod_initial)


-- Stare initiala --
    .-----------.-----------.
    |           |           |
    |   .-------.-------.   |
    |   |       |       |   |
    |   |   .---.---.   |   |
    |   |   |       |   |   |
    .---.---x       0---x---.
    |   |   |       |   |   |
    |   |   .---.---0   |   |
    |   |       |       |   |
    |   x-------0-------.   |
    |           |           |
    .-----------.-----------.

-- Stare dupa mutare --
    .-----------.-----------.
    |           |           |
    |   .-------.-------.   |
    |   |       |       |   |
    |   |   .---.---.   |   |
    |   |   |       |   |   |
    .---x---x       0---x---.
    |   |   |       |   |   |
    |   |   .---.---0   |   |
    |   |       |       |   |
    |   x-------0-------.   |
    |           |           |
    .-----------.-----------.



# Documentatie

# Justificare ponderi

Am ales sa calculez probabilistic urmatoarea mutare, calculand scorul fiecarei posibile mutari din pozitia initiala. Pentru fiecare simbol al lui MAX din moara curenta adaug 0.3 la probabilitatea totala (sau 0.1 daca nu am nici un simbol), iar daca pe moara curenta exista un simbol al lui MIN, probabilitatea va fi 0